In [3]:
# Import necessary libraries
import pandas as pd
import os
import subprocess
import json

# Define paths
data_dir = "../data"
train_file = os.path.join(data_dir, "dili_train.csv")
test_file = os.path.join(data_dir, "dili_test.csv")
valid_file = os.path.join(data_dir, "dili_valid.csv")

# Load the datasets
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)
valid_df = pd.read_csv(valid_file)

# Function to featurize SMILES using Ersilia model eos4wt0
def featurize_smiles(smiles_list, model_id="eos4wt0"):
    # Start the Ersilia service
    subprocess.run(["ersilia", "serve", model_id], check=True)
    
    # Prepare a temporary file with SMILES strings
    with open("temp_smiles.txt", "w") as f:
        for smiles in smiles_list:
            f.write(smiles + "\n")
    
    # Run Ersilia to get Morgan Fingerprints
    result = subprocess.run(
        ["ersilia", "-v", "run", "-i", "temp_smiles.txt"],
        capture_output=True,
        text=True
    )
    
    # Parse the output
    output_lines = result.stdout.splitlines()
    features = []
    for line in output_lines:
        try:
            result_dict = json.loads(line)
            feature_vector = result_dict["output"]["outcome"]
            features.append(feature_vector)
        except:
            features.append(None)  # Handle any errors 
    
    # Stop the Ersilia service
    subprocess.run(["ersilia", "close"], check=True)
    
    return features

# Featurize the SMILES strings for each dataset
train_smiles = train_df['Drug'].tolist()
test_smiles = test_df['Drug'].tolist()
valid_smiles = valid_df['Drug'].tolist()

train_features = featurize_smiles(train_smiles)
test_features = featurize_smiles(test_smiles)
valid_features = featurize_smiles(valid_smiles)

# Add features to the DataFrames
train_df['Features'] = train_features
test_df['Features'] = test_features
valid_df['Features'] = valid_df

# Check for any failed featurizations
print("Number of failed featurizations in training set:", train_df['Features'].isnull().sum())
print("Number of failed featurizations in test set:", test_df['Features'].isnull().sum())
print("Number of failed featurizations in validation set:", valid_df['Features'].isnull().sum())

# Drop rows with failed featurizations (if any)
train_df = train_df.dropna(subset=['Features'])
test_df = test_df.dropna(subset=['Features'])
valid_df = valid_df.dropna(subset=['Features'])

# Save the featurized datasets
train_df.to_csv(os.path.join(data_dir, "dili_train_featurized.csv"), index=False)
test_df.to_csv(os.path.join(data_dir, "dili_test_featurized.csv"), index=False)
valid_df.to_csv(os.path.join(data_dir, "dili_valid_featurized.csv"), index=False)

print("Featurized datasets saved successfully!")

🚀 Serving model eos4wt0: morgan-fps

   URL: http://0.0.0.0:54907
   PID: -1
   SRV: pulled_docker
   Output source: local-only

👉 To run model:
   - run

💁 Information:
   - info
⛔ Model eos4wt0 closed
🚀 Serving model eos4wt0: morgan-fps

   URL: http://0.0.0.0:45549
   PID: -1
   SRV: pulled_docker
   Output source: local-only

👉 To run model:
   - run

💁 Information:
   - info
⛔ Model eos4wt0 closed
🚀 Serving model eos4wt0: morgan-fps

   URL: http://0.0.0.0:40511
   PID: -1
   SRV: pulled_docker
   Output source: local-only

👉 To run model:
   - run

💁 Information:
   - info
⛔ Model eos4wt0 closed


ValueError: Length of values (0) does not match length of index (332)